In [20]:
### data preprocessing ###
import pandas as pd 
import numpy as np
df=pd.read_csv('/Users/hexiaoyu/Desktop/Courses/INSY- DATA MINING/cereals.CSV')
df=df.dropna()

In [21]:
#construct predictors and target variables
X=df.iloc[:,3:12]
y=df['Rating']

In [22]:
# split the training and testing dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.33, random_state= 5)

In [23]:
# standardize x
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X_train= scaler.fit_transform(X_train)
scaled_X_test= scaler.transform(X_test)

scaled_X_train = pd.DataFrame(scaled_X_train, columns=X_train.columns)
scaled_X_test = pd.DataFrame(scaled_X_test, columns=X_test.columns)

In [24]:
### model building with cross validation evaluation ###
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm. fit(scaled_X_train,y_train)
y_test_pred = model.predict(scaled_X_test)
## use mse and mape metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
lm_mse = mean_squared_error(y_test, y_test_pred)  ## (true value, the predicted value)
lm_mape = mean_absolute_percentage_error(y_test, y_test_pred)

In [25]:
print(lm_mse) 
print(lm_mape)

1.066270468838969e-13
7.437424502194683e-09


the mse and mape is closely to 0, which is good sign for modelling performance in general, but also indicating it may has overfitting issues 

the thing is we have to split the training and testing data first before standardize them as it may lead to data leakage if we scaler the whole X, making the X_test meaningless. 

In [26]:
### data preprocessing ###
import pandas as pd
df = pd.read_csv('/Users/hexiaoyu/Desktop/Courses/INSY- DATA MINING/cereals.CSV')
df = df.dropna()
#construct predictors and target variables
X = df.iloc[:, 3:12]
y = df['Rating']
# standardize the X
scaler=StandardScaler()
scaled_X= scaler.fit_transform(X)
scaled_X = pd.DataFrame(scaled_X, columns=X.columns)

In [27]:
### linear regression with k fold cv ####
lm2 = LinearRegression() 
from sklearn.model_selection import cross_validate
lm_score= cross_validate(lm2, scaled_X, y, scoring='neg_mean_squared_error',cv=5, return_estimator=True)
lm_score['test_score'].mean()
# mse scores 

np.float64(-1.217630360834181e-13)

negating MSE, it aligns with this convention (lower MSE is better in regression).

In [31]:
for model in lm_score['estimator']:
    print(pd.DataFrame(model.coef_, X.columns, columns=['Coefficients']))

          Coefficients
Calories     -4.389750
Protein       3.497412
Fat          -1.691408
Sodium       -4.479770
Fiber         8.288322
Carbo         4.222640
Sugars       -3.138475
Potass       -2.393069
Vitamins     -1.133997
          Coefficients
Calories     -4.389750
Protein       3.497412
Fat          -1.691408
Sodium       -4.479770
Fiber         8.288322
Carbo         4.222640
Sugars       -3.138475
Potass       -2.393069
Vitamins     -1.133997
          Coefficients
Calories     -4.389750
Protein       3.497412
Fat          -1.691408
Sodium       -4.479770
Fiber         8.288322
Carbo         4.222640
Sugars       -3.138475
Potass       -2.393069
Vitamins     -1.133997
          Coefficients
Calories     -4.389750
Protein       3.497412
Fat          -1.691408
Sodium       -4.479770
Fiber         8.288322
Carbo         4.222640
Sugars       -3.138475
Potass       -2.393069
Vitamins     -1.133997
          Coefficients
Calories     -4.389750
Protein       3.497412
Fat        

In [29]:
# tuning parameter k # (testing different k values)
for i in range (3,11):
    score = cross_validate(lm2, scaled_X, y, scoring='neg_mean_squared_error',cv=i, return_estimator=True)
    print("MSE using k-fold CV with ",i," folds = "+str(score['test_score'].mean()))

MSE using k-fold CV with  3  folds = -1.1242906642051351e-13
MSE using k-fold CV with  4  folds = -1.2242532774438526e-13
MSE using k-fold CV with  5  folds = -1.217630360834181e-13
MSE using k-fold CV with  6  folds = -1.1755147470947026e-13
MSE using k-fold CV with  7  folds = -1.210323331490777e-13
MSE using k-fold CV with  8  folds = -1.1682305476984324e-13
MSE using k-fold CV with  9  folds = -1.1659321169101175e-13
MSE using k-fold CV with  10  folds = -1.1376693412507738e-13


In [30]:
### LOOCV ####
lm3 = LinearRegression()
from sklearn.model_selection import LeaveOneOut
score = cross_validate(lm3, scaled_X, y, scoring='neg_mean_squared_error', cv=LeaveOneOut())
score['test_score'].mean()

np.float64(-1.1423334207926656e-13)

## Regularization :
by adding extra bias (technically by shrinking the coefficient) into training data to mitigate the overfitting issue on tranining data and improve performance

In [32]:
### ridge regression model###
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1)
ridge_model= ridge.fit(scaled_X_train,y_train)
pd.DataFrame(ridge_model.coef_, X.columns, columns=['Coefficients']) 

,Coefficients
Calories,-3.753828
Protein,2.949761
Fat,-2.635585
Sodium,-4.435950
Fiber,7.493458
Carbo,3.168961
Sugars,-4.648187
Potass,-0.983930
Vitamins,-1.198434


In [33]:
# Generate the prediction value from the test data and evaluate
y_test_pred = ridge_model.predict(scaled_X_test) 
ridge_mse = mean_squared_error(y_test, y_test_pred)

In [34]:
# Finding optimal penalty
for i in range (1,10):
    ridge = Ridge(alpha=i) #alpha: This is the regularization strength. A larger alpha imposes a stronger regularization penalty (shrinks the coefficients more), while a smaller alpha results in less regularization.
    ridge_model= ridge.fit(scaled_X_train,y_train)
    y_test_pred= ridge_model.predict(scaled_X_test)
    print('Alpha = ',i,' / MSE =',mean_squared_error(y_test, y_test_pred))

Alpha =  1  / MSE = 0.27620472715671157
Alpha =  2  / MSE = 0.64506221654001
Alpha =  3  / MSE = 1.0137810339632132
Alpha =  4  / MSE = 1.3812030054373843
Alpha =  5  / MSE = 1.750465082488685
Alpha =  6  / MSE = 2.123892303784433
Alpha =  7  / MSE = 2.5028142658319847
Alpha =  8  / MSE = 2.8878643161370494
Alpha =  9  / MSE = 3.279238168286585


In [ ]:
### Lasso regression model ###

In [35]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1)
lasso_model= lasso.fit(scaled_X_train,y_train)

In [36]:
pd.DataFrame(lasso_model.coef_, X.columns, columns=['Coefficients'])

,Coefficients
Calories,-1.097056
Protein,1.337136
Fat,-3.010541
Sodium,-3.485664
Fiber,5.783703
Carbo,0.000000
Sugars,-7.332847
Potass,0.000000
Vitamins,-0.328085


In [37]:
# Generate the prediction value from the test data and evaluate
y_test_pred = lasso_model.predict(scaled_X_test)
lasso_mse = mean_squared_error(y_test, y_test_pred)

In [38]:
# Finding optimal alpha
for i in range (1,10):
    lasso = Lasso(alpha=i)
    lasso_model = lasso.fit(scaled_X_train,y_train)
    y_test_pred = lasso_model.predict(scaled_X_test)
    print('Alpha = ',i,' / MSE =',mean_squared_error(y_test, y_test_pred))

Alpha =  1  / MSE = 4.666647066791286
Alpha =  2  / MSE = 14.217247294191658
Alpha =  3  / MSE = 30.05185989224861
Alpha =  4  / MSE = 46.394886081731144
Alpha =  5  / MSE = 58.297877798041654
Alpha =  6  / MSE = 70.7268532702123
Alpha =  7  / MSE = 85.09439478685343
Alpha =  8  / MSE = 101.40058665573137
Alpha =  9  / MSE = 113.88364027671432


## Regularization on logistic regression

In [39]:
df['binary_rating'] = df['Rating'].apply(lambda x: 1 if x > 50 else 0)
y = df['binary_rating']

# Re-split with new binary target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 5)

In [40]:
from sklearn.linear_model import LogisticRegression
Ir = LogisticRegression(penalty='l1', solver='liblinear') # l1 refers to lasso , l2 is ridge
model_Ir = Ir.fit(scaled_X_train,y_train)
y_test_pred = model_Ir.predict(scaled_X_test)

In [41]:
# Get performance measures
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_test_pred)
precision = metrics.precision_score(y_test, y_test_pred)
recall = metrics.recall_score(y_test, y_test_pred)
f1 = metrics.f1_score(y_test, y_test_pred)
print("Accuracy score = "+str(accuracy))
print("Precision score = "+str(precision))
print("Recall score = "+str(recall))
print("F1 score = "+str(f1)) # balancing precision and recall

Accuracy score = 0.96
Precision score = 0.75
Recall score = 1.0
F1 score = 0.8571428571428571


In [ ]:
### the library used 
# data preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# model selection 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#cross validate k-fold and loocv 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut

# performance score 
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error # linear regression 
from sklearn import metrics # logistic regression 